# Práctica guiada: Patrón MVC en Flask (Ejercicios)


Este cuaderno está diseñado para practicar **MVC en Flask** siguiendo las ideas de la presentación *“Patrones de Diseño MVC en Flask”* (separación de responsabilidades, evitar “todo en app.py”, y errores típicos).  

Objetivo: crear una mini‑app **“ToDo”** con estructura MVC, usando:
- **Modelo**: clases + validaciones/reglas
- **Controlador**: rutas + coordinación
- **Vista**: templates “tontos” (Jinja2 solo para mostrar)

> Nota: en Jupyter no se suele dejar un servidor corriendo en la misma celda. Ejecutaremos comandos para crear archivos y luego arrancaremos Flask desde terminal.

## Requisitos
- Python 3.10+ (o similar)
- `flask` instalado

Instalación rápida:
```bash
python -m venv .venv
# Windows: .venv\Scripts\activate
# Linux/Mac: source .venv/bin/activate
pip install flask
```

## Estructura del proyecto (MVC)
Vamos a crear esta estructura (con `controllers/`, `models/`, `templates/`):

```
mvc_todo/
├─ app.py
├─ controllers/
│  ├─ __init__.py
│  └─ todo_controller.py
├─ models/
│  ├─ __init__.py
│  └─ todo_model.py
├─ templates/
│  ├─ base.html
│  ├─ index.html
│  └─ todo_form.html
└─ static/
   └─ css/
      └─ style.css
```

**Regla didáctica** (de la presentación):  
- `models/` = datos + reglas  
- `controllers/` = coordinar y flujo  
- `templates/` = presentación pura

## 1) Crear carpetas del proyecto
Ejecuta esta celda una sola vez.

In [1]:
import os, pathlib

ROOT = pathlib.Path("mvc_todo")
paths = [
    ROOT / "controllers",
    ROOT / "models",
    ROOT / "templates",
    ROOT / "static" / "css",
]
for p in paths:
    p.mkdir(parents=True, exist_ok=True)

print("✅ Estructura creada en:", ROOT.resolve())


✅ Estructura creada en: C:\Users\Puesto04\DALP\PROO\Flask\mvc_todo


## 2) Crear `app.py` (solo arranque + registro de controladores)
**Ejercicio**: observa que `app.py` no debe contener lógica de negocio.

In [2]:
%%writefile mvc_todo/app.py
from flask import Flask
from controllers.todo_controller import todo_bp

def create_app() -> Flask:
    app = Flask(__name__)
    app.config["SECRET_KEY"] = "dev"  # en producción, usa variable de entorno

    # Registro del controlador (Blueprint)
    app.register_blueprint(todo_bp)

    return app

if __name__ == "__main__":
    app = create_app()
    app.run(debug=True)


Writing mvc_todo/app.py


## 3) Modelo `Todo` + reglas (models)
Las **validaciones y reglas** deben vivir en el Modelo, no en los templates. fileciteturn0file0

In [3]:
%%writefile mvc_todo/models/todo_model.py
from dataclasses import dataclass, field
from datetime import datetime

class ValidationError(ValueError):
    pass

@dataclass
class Todo:
    title: str
    done: bool = False
    created_at: datetime = field(default_factory=datetime.utcnow)

    def validate(self) -> None:
        # Regla de negocio: título obligatorio y con longitud mínima
        if not isinstance(self.title, str):
            raise ValidationError("El título debe ser texto.")
        clean = self.title.strip()
        if len(clean) < 3:
            raise ValidationError("El título debe tener al menos 3 caracteres.")
        self.title = clean

class TodoRepository:
    \"\"\"Repositorio simple en memoria (para practicar MVC sin BD).\"\"\"
    def __init__(self):
        self._items: list[Todo] = []

    def list_all(self) -> list[Todo]:
        return list(self._items)

    def add(self, todo: Todo) -> None:
        todo.validate()
        self._items.append(todo)

    def toggle(self, index: int) -> None:
        if index < 0 or index >= len(self._items):
            raise ValidationError("Índice fuera de rango.")
        self._items[index].done = not self._items[index].done

    def delete(self, index: int) -> None:
        if index < 0 or index >= len(self._items):
            raise ValidationError("Índice fuera de rango.")
        self._items.pop(index)


Writing mvc_todo/models/todo_model.py


In [4]:
%%writefile mvc_todo/models/__init__.py
from .todo_model import Todo, TodoRepository, ValidationError

__all__ = ["Todo", "TodoRepository", "ValidationError"]


Writing mvc_todo/models/__init__.py


## 4) Controlador (routes/blueprint)
**Ejercicio**: el controlador debe ser *delgado* (coordina, delega al Modelo). fileciteturn0file0

In [5]:
%%writefile mvc_todo/controllers/todo_controller.py
from flask import Blueprint, render_template, request, redirect, url_for, flash
from models import Todo, TodoRepository, ValidationError

todo_bp = Blueprint("todo", __name__)

# Repositorio en memoria (para el ejercicio). En un proyecto real lo inyectarías.
repo = TodoRepository()

@todo_bp.get("/")
def index():
    items = repo.list_all()
    return render_template("index.html", items=items)

@todo_bp.get("/nuevo")
def new_form():
    return render_template("todo_form.html")

@todo_bp.post("/nuevo")
def create():
    title = request.form.get("title", "")
    try:
        repo.add(Todo(title=title))
        flash("Tarea creada ✅", "success")
        return redirect(url_for("todo.index"))
    except ValidationError as e:
        flash(str(e), "error")
        return render_template("todo_form.html", title=title), 400

@todo_bp.post("/toggle/<int:index>")
def toggle(index: int):
    try:
        repo.toggle(index)
        return redirect(url_for("todo.index"))
    except ValidationError as e:
        flash(str(e), "error")
        return redirect(url_for("todo.index")), 400

@todo_bp.post("/delete/<int:index>")
def delete(index: int):
    try:
        repo.delete(index)
        flash("Tarea eliminada 🗑️", "success")
        return redirect(url_for("todo.index"))
    except ValidationError as e:
        flash(str(e), "error")
        return redirect(url_for("todo.index")), 400


Writing mvc_todo/controllers/todo_controller.py


In [6]:
%%writefile mvc_todo/controllers/__init__.py
# Paquete de controladores


Writing mvc_todo/controllers/__init__.py


## 5) Vistas (templates)
Recuerda: templates “tontos” (presentación). No metas lógica de negocio aquí. fileciteturn0file0

In [7]:
%%writefile mvc_todo/templates/base.html
<!doctype html>
<html lang="es">
<head>
  <meta charset="utf-8">
  <title>{% block title %}MVC ToDo{% endblock %}</title>
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <link rel="stylesheet" href="{{ url_for('static', filename='css/style.css') }}">
</head>
<body>
  <header>
    <h1><a href="{{ url_for('todo.index') }}">MVC ToDo</a></h1>
    <nav>
      <a href="{{ url_for('todo.new_form') }}">Nueva tarea</a>
    </nav>
  </header>

  <section class="messages">
    {% with messages = get_flashed_messages(with_categories=true) %}
      {% if messages %}
        <ul>
          {% for category, msg in messages %}
            <li class="{{ category }}">{{ msg }}</li>
          {% endfor %}
        </ul>
      {% endif %}
    {% endwith %}
  </section>

  <main>
    {% block content %}{% endblock %}
  </main>
</body>
</html>


Writing mvc_todo/templates/base.html


In [8]:
%%writefile mvc_todo/templates/index.html
{% extends "base.html" %}
{% block title %}Listado{% endblock %}

{% block content %}
  {% if items|length == 0 %}
    <p>No hay tareas todavía. Crea la primera.</p>
  {% else %}
    <ul class="todo-list">
      {% for t in items %}
        <li class="{% if t.done %}done{% endif %}">
          <span>{{ t.title }}</span>
          <form method="post" action="{{ url_for('todo.toggle', index=loop.index0) }}">
            <button type="submit">{% if t.done %}Desmarcar{% else %}Hecha{% endif %}</button>
          </form>
          <form method="post" action="{{ url_for('todo.delete', index=loop.index0) }}">
            <button type="submit">Borrar</button>
          </form>
        </li>
      {% endfor %}
    </ul>
  {% endif %}
{% endblock %}


Writing mvc_todo/templates/index.html


In [9]:
%%writefile mvc_todo/templates/todo_form.html
{% extends "base.html" %}
{% block title %}Nueva tarea{% endblock %}

{% block content %}
  <h2>Nueva tarea</h2>
  <form method="post" action="{{ url_for('todo.create') }}">
    <label for="title">Título</label>
    <input id="title" name="title" type="text" value="{{ title|default('') }}" required>
    <button type="submit">Crear</button>
  </form>

  <p class="hint">Regla: mínimo 3 caracteres (validado en el Modelo).</p>
{% endblock %}


Writing mvc_todo/templates/todo_form.html


## 6) Estáticos (CSS)
Si ves *Not Found* en `/static/...`, revisa rutas y carpeta `static/`.

In [10]:
%%writefile mvc_todo/static/css/style.css
body { font-family: system-ui, Arial, sans-serif; margin: 20px; }
header { display: flex; align-items: baseline; gap: 16px; }
header a { text-decoration: none; }
.todo-list { padding-left: 0; list-style: none; }
.todo-list li { display: flex; gap: 8px; align-items: center; margin: 8px 0; }
.todo-list li.done span { text-decoration: line-through; opacity: 0.7; }
.messages ul { padding-left: 0; list-style: none; }
.messages li.success { background: #eaffea; padding: 8px; }
.messages li.error { background: #ffecec; padding: 8px; }
.hint { font-size: 0.9rem; opacity: 0.8; }


Writing mvc_todo/static/css/style.css


## 7) Ejecución
Arranca Flask desde terminal (en la carpeta donde está `mvc_todo/`).

In [11]:
print("En terminal:")
print("  cd mvc_todo")
print("  python app.py")
print("y abre: http://127.0.0.1:5000/")


En terminal:
  cd mvc_todo
  python app.py
y abre: http://127.0.0.1:5000/


## 8) EJERCICIOS (para practicar MVC)
Completa cada ejercicio **sin romper la separación de responsabilidades**. fileciteturn0file0

### Ejercicio 1 — Añadir prioridad (Modelo)
1. Añade un campo `priority` a `Todo` (valores: `low`, `medium`, `high`).
2. Valida en el **Modelo** que solo se acepten esos 3 valores.
3. Muestra la prioridad en la vista `index.html`.

**Pista**: añade un `<select>` en `todo_form.html`. La validación NO debe estar en el template.

In [ ]:
from dataclasses import field
from datetime import datetime

from flask import flash, redirect, render_template, request, url_for

from PROO.Flask.mvc_todo.models.todo_model import ValidationError

# Modelo
class Todo:
    title: str
    priority: str
    done: bool = False
    created_at: datetime = field(default_factory=datetime.utcnow)

    def validate(self) -> None:
        # Regla de negocio: título obligatorio y con longitud mínima
        if not isinstance(self.title, str):
            raise ValidationError("El título debe ser texto.")
        clean = self.title.strip()
        if len(clean) < 3:
            raise ValidationError("El título debe tener al menos 3 caracteres.")
        if self.priority not in ("low", "medium", "high"):
            raise ValidationError(
                "El valor de prioridad no coincide con los aceptados."
            )
        self.title = clean

# Controlador
@todo_bp.post("/nuevo") # type: ignore
def create():
    title = request.form.get("title", "")
    priority = request.form.get("priority", "")
    try:
        repo.add(Todo(title=title, priority=priority)) # type: ignore
        flash("Tarea creada ✅", "success")
        return redirect(url_for("todo.index"))
    except ValidationError as e:
        flash(str(e), "error")
        return render_template("todo_form.html", title=title, priority=priority), 400
    
# Vista todo_form.html
{% extends "base.html" %}
{% block title %}Nueva tarea{% endblock %}

{% block content %}
<h2>Nueva tarea</h2>
<form method="post" action="{{ url_for('todo.create') }}">
  <label for="title">Título</label>
  <input id="title" name="title" type="text" value="{{ title|default('') }}" required>
  <select name="priority">
    <option value="low">Baja</option>
    <option value="medium">Media</option>
    <option value="high">Alta</option>
  </select>
  <button type="submit">Crear</button>
</form>

<p class="hint">Regla: mínimo 3 caracteres (validado en el Modelo).</p>
{% endblock %}

### Ejercicio 2 — Regla de negocio: evitar duplicados (Modelo)
1. Modifica `TodoRepository.add()` para que no permita dos tareas con el mismo título (ignorando mayúsculas/minúsculas).
2. Si hay duplicado, lanza `ValidationError("Ya existe una tarea con ese título.")`.

**Objetivo**: que el controlador no tenga que saber cómo se decide si es duplicado.

In [ ]:
# Modelo TodoRepository
def add(self, todo: Todo) -> None:
        todo.validate()
        for item in self._items:
            if item.title.lower() == todo.title.lower():
                raise ValidationError("Tarea duplicada.")
        self._items.append(todo)

### Ejercicio 3 — Controlador delgado: extraer “servicio”
Crea un módulo `models/todo_service.py` con una clase `TodoService` que use el repositorio y exponga:
- `create_todo(title: str) -> None`
- `toggle_todo(index: int) -> None`
- `delete_todo(index: int) -> None`

Después, modifica el controlador para usar el servicio y que cada ruta tenga **pocas líneas**.

**Criterio**: si tus rutas empiezan a tener “muchas decisiones”, muévelo al Modelo/Servicio.

### Ejercicio 4 — Vista “tonta”: prohibido calcular en Jinja
En `index.html`, evita cualquier lógica que no sea de presentación.  
Ejemplos de “mala práctica” (de la presentación):
- cálculos complejos
- validaciones
- decisiones de negocio

**Tarea**: si quieres mostrar “Tareas completadas: X/Y”, calcula ese dato en el controlador (o servicio) y pásalo al template.

### Ejercicio 5 — Ruta `/saludo/<nombre>` (MVC mínimo)
Implementa una nueva ruta:
- URL: `/saludo/<nombre>`
- Controlador: recibe `nombre` y delega una función del Modelo para validarlo/formatearlo (p.ej. capitalizar, rechazar números)
- Vista: template `saludo.html` con el mensaje final

**Objetivo**: replicar el flujo explicado en la presentación (usuario → controlador → modelo → vista). fileciteturn0file0

### Ejercicio 6 — Sustituir “memoria” por JSON (Modelo)
Cambia el repositorio para persistir en un archivo `data/todos.json`:
- `list_all()` lee el JSON
- `add/toggle/delete` escriben el JSON

**Regla**: el controlador NO toca archivos. Solo llama al Modelo/Repositorio.

### Ejercicio 7 — Bonus: tests de Modelo (sin Flask)
Crea tests (con `pytest` o `unittest`) para:
- `Todo.validate()` (títulos cortos, espacios, tipos)
- `TodoRepository` (toggle y delete fuera de rango)

**Idea**: el Modelo debería poder testearse sin servidor web.